# Prepare dataset for training and validation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../images/ciffer/train',  # this is the target directory
        target_size=(112, 112),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '../images/ciffer/validation',
        target_size=(112, 112),
        batch_size=batch_size,
        class_mode='categorical')

# Create the model

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 512, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using TensorFlow backend.


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 510, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 126, 510, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 255, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 253, 32)       9248      
_________________________________________________________________
activation_2 (Activation)    (None, 61, 253, 32)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 126, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 124, 32)       9248      
__________

# Train the network

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1600 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=16  // batch_size)

In [ ]:
model.save_weights('../models/ciffer.h5')

# Test the network with unknown data

In [ ]:
batch_size = 16
test_generator = test_datagen.flow_from_directory(
        '../images/ciffer/test',
        target_size=(112, 112),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
model.evaluate_generator(
        test_generator,
        steps=16 // batch_size)